# Imports...

In [1]:
# disable warnings bc we good on this one
import warnings
warnings.filterwarnings('ignore')

import os
from pprint import pprint
import numpy as np
import pandas as pd
import pickle
from tqdm import tqdm_notebook as tqdm
from scipy.io import loadmat


from datetime import datetime

from pynwb import NWBFile, TimeSeries, NWBHDF5IO
from pynwb.file import Subject

# install manually with python ./setup.py install
# https://github.com/NeurodataWithoutBorders/nwb-jupyter-widgets
#from nwbwidgets import nwb2widget

# local version of oe's analysis-tools
import OpenEphys as oe

data_dir = '../data/nwb/'




# Loading...

Load our own idiosyncratic metadata format. It is stored as a matlab struct, and contains sorted spikes as well as metadata.

In [2]:
out = loadmat(os.path.join(data_dir, 'OutPSTH_ch2c1.mat'))
out = out['out']

Scipy loads matlab structs as record arrays, to see the fields we can refer to its dtype

In [3]:
out.dtype.names

('IL',
 'Nclusters',
 'tetrode',
 'channel',
 'cluster',
 'cell',
 'M1ON',
 'mM1ONspikecount',
 'sM1ONspikecount',
 'semM1ONspikecount',
 'mM1ON',
 'mM1spontON',
 'sM1spontON',
 'semM1spontON',
 'LaserStart',
 'LaserWidth',
 'Lasernumpulses',
 'M_LaserStart',
 'M_LaserWidth',
 'M_LaserNumPulses',
 'M_LaserISI',
 'M1OFF',
 'mM1OFF',
 'mM1OFFspikecount',
 'sM1OFFspikecount',
 'semM1OFFspikecount',
 'mM1spontOFF',
 'sM1spontOFF',
 'semM1spontOFF',
 'numpulseamps',
 'numgapdurs',
 'pulseamps',
 'gapdurs',
 'gapdelay',
 'soa',
 'nrepsON',
 'nrepsOFF',
 'xlimits',
 'samprate',
 'datadir',
 'spiketimes',
 'LaserRecorded',
 'StimRecorded',
 'M1ONLaser',
 'mM1ONLaser',
 'M1OFFLaser',
 'mM1OFFLaser',
 'M1ONStim',
 'mM1ONStim',
 'M1OFFStim',
 'mM1OFFStim',
 'nb',
 'stimlog',
 'user')

Wow that's pretty big list of opaque variable names. Thankfully that's the point of converting to NWB.

We are only going to work with a small subset of it anyway so don't get your peppers pickled.

## Stimulus Metadata

Extract recording metadata. First we get the stim log, we convert it to a pandas DataFrame so it's easier to work with, and do a bit of cleaning.

In [4]:
stim_log = pd.DataFrame.from_records(out['stimlog'][0][0][0])

# remove columns we don't need
stim_log = stim_log[['type', 'stimulus_description', 'timestamp', 'LaserOnOff']]

stim_log = stim_log[stim_log.columns].astype(str)

# remove leading and lagging brackets
for name, col in stim_log.iteritems():
    stim_log.loc[:,name] = col.str.lstrip("[u'").str.rstrip("]'")

# clean up timestamps
stim_log.loc[:,'timestamp'] = stim_log['timestamp'] + "000"
stim_log.loc[:,'timestamp'] = pd.to_datetime(stim_log['timestamp'], 
                                             format="%B-%d-%Y %H:%M:%S.%f")


Parse this PITA... i mean plaintext stim description... it looks like this:

In [5]:
print(stim_log.loc[0:5, 'stimulus_description'])

0    whitenoise amplitude:80 ramp:0 loop_flg:0 seam...
1    whitenoise amplitude:80 ramp:0 loop_flg:0 seam...
2    whitenoise amplitude:80 ramp:0 loop_flg:0 seam...
3    whitenoise amplitude:80 ramp:0 loop_flg:0 seam...
4    whitenoise amplitude:80 ramp:0 loop_flg:0 seam...
5    GPIAS amplitude:80 ramp:0 soa:50 soaflag:isi l...
Name: stimulus_description, dtype: object


so we need to split that first by spaces and then by colons to break it up into new columns.

In [6]:
# Expand it to columns

stim_desc = stim_log.loc[:,'stimulus_description'].str.split(' ',expand=True)

# two types of stim have two sets of params, do white noise first
stim_noise = stim_desc.loc[stim_desc[0]=='whitenoise']
col_names = ['type']

# iterate through columns, splitting on colons and 
# assigning the left to the name and the right as the value
for i in range(1,stim_noise.shape[1]):
    try:
        split_col = stim_noise.loc[:,i].str.split(':', expand=True)
        # get name of column
        col_name = split_col[0][0]
        if col_name is None:
            continue
        col_names.append(col_name)
        # replace column w value
        stim_noise.loc[:,i] = split_col[1]
    except:
        pass
    
# remove None columns and set names
stim_noise = stim_noise.loc[:,0:len(col_names)-1]
stim_noise.columns = col_names


In [7]:

#######
# YEAH REPEATING OURSELVES
########


stim_gpias = stim_desc.loc[stim_desc[0]=='GPIAS']
col_names = ['type']
for i in range(1,stim_gpias.shape[1]):
    try:
        split_col = stim_gpias.loc[:,i].str.split(':', expand=True)
        # get name of column
        col_name = split_col.iloc[0,0]
        if col_name is None:
            continue
        col_names.append(col_name)
        # replace column w value
        stim_gpias.loc[:,i] = split_col[1]
    except:
        pass
    
# remove None columns and set names
stim_gpias = stim_gpias.loc[:,0:len(col_names)-1]
stim_gpias.columns = col_names
    

In [8]:
# join them by making a fake index column to preserve position later
stim_noise['idx'] = stim_noise.index
stim_gpias['idx'] = stim_gpias.index
stim_combined = stim_noise.merge(stim_gpias, how="outer")
stim_combined.loc[:,'idx'] = stim_combined['idx'].astype('int64') 

Now we have some sensible looking stimulus parameters, but they aren't sorted yet.

In [9]:
stim_combined.head(10)

,type,amplitude,ramp,loop_flg,seamless,duration,next,idx,soa,soaflag,gapdelay,gapdur,pulsedur,pulseamp,laser
0,whitenoise,80,0,0,1,2331,-500,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,whitenoise,80,0,0,1,2331,-500,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,whitenoise,80,0,0,1,2331,-500,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,whitenoise,80,0,0,1,2331,-500,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,whitenoise,80,0,0,1,2331,-500,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,whitenoise,80,0,0,1,2331,-500,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,whitenoise,80,0,0,1,2331,-500,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,whitenoise,80,0,0,1,2331,-500,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,whitenoise,80,0,0,1,2331,-500,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,whitenoise,80,0,0,1,2331,-500,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We now recombine the stimulus details with the rest of the stimulus log, and...

In [10]:
# and recombine with main stim log
stim_log['idx'] = stim_log.index
stim_df = stim_log.merge(stim_combined, how="outer")
stim_df.set_index('idx', inplace=True)
stim_df.sort_index(inplace=True)

# drop old column
stim_df.drop('stimulus_description',axis=1, inplace=True)

The final stimulus dataframe:

In [11]:
stim_df.head(10)

,type,timestamp,LaserOnOff,amplitude,ramp,loop_flg,seamless,duration,next,soa,soaflag,gapdelay,gapdur,pulsedur,pulseamp,laser
idx,,,,,,,,,,,,,,,,
0,whitenoise,2019-03-04 04:46:10.329,0,80,0,0,1,2331,-500,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,whitenoise,2019-03-04 04:46:12.304,0,80,0,0,1,2331,-500,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,whitenoise,2019-03-04 04:46:14.209,0,80,0,0,1,2331,-500,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,whitenoise,2019-03-04 04:46:16.123,0,80,0,0,1,2331,-500,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,whitenoise,2019-03-04 04:46:18.024,0,80,0,0,1,2331,-500,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,GPIAS,2019-03-04 04:46:19.940,0,80,0,0,1,2331,-500,50,isi,1000,256,25,100,0
6,whitenoise,2019-03-04 04:46:21.842,0,80,0,0,1,2331,-500,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,whitenoise,2019-03-04 04:46:23.746,0,80,0,0,1,2331,-500,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,whitenoise,2019-03-04 04:46:25.661,0,80,0,0,1,2331,-500,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Recording Metadata & Spiketimes

Next we'll get metadata that's particular to this recording...

In [12]:
# fieldnames are in the dtype attribute
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.io.loadmat.html
field_names = out.dtype.names

get_fields = ['tetrode', 'channel', 'cluster', 'cell',
              'spiketimes']
out_dict = {}
for field in get_fields:
    # for some reason everything is nested 3 arrays deep...
    # I'm going to hardcode this, but you could just keep digging until you get an array that isn't size 1
    field_data = out[field][0][0][0]
    
    # if this is an experiment-wide piece of metadata, 
    # get it out of the array
    if field_data.shape == (1,):
        field_data = field_data[0]
        
    out_dict[field] = field_data
    


In [13]:
pprint(out_dict)

{'cell': 1,
 'channel': 2,
 'cluster': 1,
 'spiketimes': array([  20.9655,   44.8939,   51.1769,   59.727 ,   61.1946,   67.7939,
         82.6213,   97.8639,   98.7214,  107.901 ,  108.0083,  108.318 ,
        110.9303,  124.7441,  124.7567,  124.8284,  125.357 ,  125.3916,
        125.5729,  134.9357,  134.9821,  139.8962,  146.2673,  147.5681,
        161.2608,  161.2692,  174.3744,  176.3128,  176.3674,  177.988 ,
        178.0086,  178.2294,  178.5374,  179.2267,  179.832 ,  198.2626,
        199.1443,  199.2332,  199.2403,  199.8766,  201.6041,  202.1028,
        207.1975,  208.2051,  237.3535,  243.4475,  252.2219,  289.0311,
        290.4128,  307.6082,  307.6908,  307.7069,  328.7144,  328.766 ,
        329.1264,  329.2594,  329.2658,  347.2154,  348.9669,  363.6879,
        368.3297,  398.0272,  408.5268,  408.5354,  444.4572,  461.1884,
        482.9897,  497.9009,  497.9217,  506.5821,  509.7768,  520.3779,
        520.4915,  523.6779,  536.2337,  549.3541,  560.9855,  566.

## Experiment Metadata

and finally the metadata that's particular to the whole experiment. This is stored in a separate notebook file. So yes, we have to parse more matlab structs, but thankfull this time it's less painful (although nothing in MATLAB is pain*less*).

In [14]:
# We also have a separate notebook file...
notebook = loadmat(os.path.join(data_dir, 'notebook.mat'))

notebook_fields = ['user', 'mouseID', 'mouseDOB', 'mouseSex', 'mouseGenotype', 'Drugs']

expt_dict = {}
for field in notebook_fields:
    expt_dict[field] = notebook['nb'][field][0][0][0]
    

In [15]:
pprint(expt_dict)

{'Drugs': u'none',
 'mouseDOB': u'11/28/18',
 'mouseGenotype': u'100012',
 'mouseID': u'9083',
 'mouseSex': u'f',
 'user': u'Kat'}


## Raw Ephys Data

We have saved our raw data as OpenEphys `.continuous` data, use the [analysis-tools](https://github.com/open-ephys/analysis-tools) to load it. I have made some fixes to this code, it being buggy as all hell, and so am using a local version.

I am not including the raw .continuous files in this demo because they are ~560MB, but the following was what was used to load, truncate, and add them to the NWB file.

In [16]:
# oe_files = [os.path.join(data_dir, fn)
#             for fn in os.listdir(data_dir) 
#             if fn.endswith('continuous')]
# 
# continuous = []
# 
# #oe_evts = oe.load(os.path.join(data_dir,'all_channels.events'))
# 
# 
# for fn in tqdm(oe_files):
#     oe_data = oe.load(fn)
#     # construct dict to store data
#     oe_dict = {
#         'name': oe_data['header']['channel'].decode().strip("'"),
#         'fs' : oe_data['header']['sampleRate'],
#         'scale': oe_data['header']['bitVolts'],
#         'data': oe_data['data'][0:1000]
#     }
#     
#     continuous.append(oe_dict)
    
cont_fn = os.path.join(data_dir, 'continuous_dict.pck')
# with open(cont_fn, 'wb+') as cont_file:
#     pickle.dump(continuous, cont_file)
    
with open(cont_fn, 'rb') as cont_file:
    continuous = pickle.load(cont_file)
    


In [17]:
pprint(continuous[0])

{'data': array([ -42.705,  -39.78 ,  -37.05 ,  -34.515,  -31.98 ,  -29.835,
        -28.08 ,  -26.52 ,  -25.35 ,  -24.57 ,  -24.18 ,  -23.985,
        -24.375,  -24.96 ,  -26.13 ,  -27.495,  -29.25 ,  -31.395,
        -33.93 ,  -36.66 ,  -39.585,  -42.705,  -45.825,  -48.75 ,
        -51.87 ,  -54.99 ,  -58.11 ,  -60.84 ,  -63.57 ,  -66.3  ,
        -68.64 ,  -70.98 ,  -73.125,  -75.27 ,  -77.415,  -79.56 ,
        -81.51 ,  -83.07 ,  -84.825,  -86.19 ,  -87.555,  -88.725,
        -89.7  ,  -90.48 ,  -91.065,  -91.455,  -91.65 ,  -91.65 ,
        -91.845,  -92.04 ,  -92.235,  -92.625,  -93.015,  -93.6  ,
        -93.99 ,  -94.575,  -95.355,  -96.135,  -96.915,  -97.89 ,
        -98.67 ,  -99.645, -100.425, -101.205, -101.985, -102.96 ,
       -103.74 , -104.52 , -105.495, -106.275, -107.055, -107.64 ,
       -108.42 , -108.81 , -109.2  , -109.395, -109.2  , -109.005,
       -108.81 , -108.42 , -108.03 , -107.445, -106.86 , -106.275,
       -105.495, -104.52 , -103.74 , -102.96 , -101.9

# Create the .nwb File

Following the [example](https://pynwb.readthedocs.io/en/stable/tutorials/general/file.html#sphx-glr-tutorials-general-file-py)

In [18]:
# describe our mouse!
this_mouse = Subject(subject_id =expt_dict['mouseID'],
                  date_of_birth= datetime.strptime(expt_dict['mouseDOB'], '%m/%d/%y'),
                  genotype   = expt_dict['mouseGenotype'],
                  sex = expt_dict['mouseSex'],
                  species = 'mouse')

create_date = datetime.now()

# use the first stimulus timestamp as start time
start_time = stim_log['timestamp'][0]

nwbfile = NWBFile('example nwb conversion', 'NWB123', start_time,
                  file_create_date=create_date,
                  experimenter=expt_dict['user'],
                  subject=this_mouse)

Set some generic metadata

In [19]:
nwbfile.lab = 'Wehr Lab'
nwbfile.institution = 'University of Oregon'
nwbfile.pharmacology = expt_dict['Drugs']

## Add Ephys Timeseries

Add our raw timeseries from the four `.continuous` files we parsed above. Normally we also collect the (sound) stimulus on an unused input channel, so rather than `.add_acquisition` we would use `.add_stimulus` for those, but I'm omitting here for brevity.

Rather than explicit timestamps, we pass `starting_time` and `rate` because this data is sampled regularly.

In [20]:
for cont in continuous:
    nwbfile.add_acquisition(
        TimeSeries(cont['name'],     cont['data'],   'V',
                  starting_time=0.0, rate=cont['fs'],
                  conversion=cont['scale'])
    )

## Add Unit

We add the spiketimes from the clustered cell here. We first have to use `.add_unit_column` to declare the params we will be saving, and then we add the unit. 

In [21]:
# Add descriptions to our new fields
add_fields = [('tetrode', 'the tetrode number this cell was recorded from'),
              ('channel', 'the recording channel \(within the tetrode\) that this cell has the strongest amplitude on'),
              ('cluster', 'the cluster number of this cell within a tetrode')]

for field in add_fields:
    nwbfile.add_unit_column(field[0], field[1])
    

In [22]:
# Then add the data -- we only have one cell in this demo
nwbfile.add_unit(id=int(out_dict['cell']), 
                 spike_times=out_dict['spiketimes'],
                 tetrode = out_dict['tetrode'],
                 channel = out_dict['channel'],
                 cluster = out_dict['cluster'])

# Add Stimulus Epochs

We use the `.add_trial` method to add discrete, trial-level information about our data in addition to the raw stim traces we should have attached.


In [23]:
# first make the trial columns
trial_descriptors = [
    ('type',      'stimulus type'),
    ('amplitude', 'stimulus amplitude in dB'),
    ('ramp',      'stimulus onset offset ramp'),
    ('loop_flg',  'whether the stimulus loops'),
    ('seamless',  'whether the loop is seamless'),
    ('duration',  'duration of stimulus'),
    ('next',      'time to next stimulus'),
    ( 'soa',      'stimulus onset asynchrony'),
    ('soaflag',   'type of soa, isi or iti'),
    ('gapdelay',  'delay until gap onset'),
    ('gapdur',    'duration of gap'),
    ('pulsedur',  'duration of noise pulse'),
    ('pulseamp',  'amplitude of noise pulse'),
    ('laser',     'laser on or off'),
    ('LaserOnOff','laser on or off again for some reason')
    ]

for descriptor in trial_descriptors:
    nwbfile.add_trial_column(descriptor[0], descriptor[1])

In [24]:
# Then add the trials

# make timestamps into time deltas

time_delts = stim_df['timestamp'] - start_time
time_delts = time_delts.dt.total_seconds()

# replace all NaNs
stim_df.fillna(value='', inplace=True)

# make timedel

for i in range(stim_df.shape[0]-1):
    nwbfile.add_trial(
        start_time = time_delts.loc[i],
        stop_time  = time_delts.loc[i+1],
        type       = stim_df.loc[i, 'type'],
        amplitude  = stim_df.loc[i, 'amplitude'],
        ramp       = stim_df.loc[i, 'ramp'],
        loop_flg   = stim_df.loc[i, 'loop_flg'],
        seamless   = stim_df.loc[i, 'seamless'],
        duration   = stim_df.loc[i, 'duration'],
        next       = stim_df.loc[i, 'next'],
        soa        = stim_df.loc[i, 'soa'],
        soaflag    = stim_df.loc[i, 'soaflag'],
        gapdelay   = stim_df.loc[i, 'gapdelay'],
        gapdur     = stim_df.loc[i, 'gapdur'],
        pulsedur   = stim_df.loc[i, 'pulsedur'],
        pulseamp   = stim_df.loc[i, 'pulseamp'],
        laser      = stim_df.loc[i, 'laser'],
        LaserOnOff = stim_df.loc[i, 'LaserOnOff'])


# Save the file

In [25]:
io = NWBHDF5IO('convert_example.nwb', mode='w')
io.write(nwbfile)
io.close()

# Viola!

Welcome to the future...
